# Gesture

### Importing Neccessary Libraries

In [1]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


### Image Data Agumentation

In [4]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
1test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [5]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'C:\humanfacesdata\trainset',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'C:\humanfacesdata\testset',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical') 

Found 360 images belonging to 4 classes.
Found 360 images belonging to 4 classes.


In [6]:
print(x_train.class_indices)#checking the number of classes

{'allu arjun': 0, 'modi': 1, 'obama': 2, 'selena': 3}


In [7]:
print(x_test.class_indices)#checking the number of classes

{'allu arjun': 0, 'modi': 1, 'obama': 2, 'selena': 3}


In [8]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 79, 1: 83, 2: 98, 3: 100})

### Creating the model

In [9]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 5

### Compiling the model

In [11]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [12]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=5, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Epoch 1/5
72/72 [==============================] - 27s 376ms/step - loss: 1.2811 - acc: 0.4361 - val_loss: 1.2170 - val_acc: 0.4889
Epoch 2/5
72/72 [==============================] - 10s 145ms/step - loss: 0.9014 - acc: 0.6444 - val_loss: 0.6127 - val_acc: 0.7944
Epoch 3/5
72/72 [==============================] - 9s 123ms/step - loss: 0.7342 - acc: 0.7139 - val_loss: 0.5724 - val_acc: 0.7833
Epoch 4/5
72/72 [==============================] - 9s 120ms/step - loss: 0.5877 - acc: 0.7694 - val_loss: 0.4260 - val_acc: 0.8250
Epoch 5/5
72/72 [==============================] - 8s 112ms/step - loss: 0.5428 - acc: 0.8111 - val_loss: 0.3393 - val_acc: 0.9139


### Saving our model

In [13]:
# Save the model
classifier.save('gesture.h5')

In [14]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

### Predicting our results

In [17]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("gesture.h5") #loading the model for testing

In [18]:
import numpy as np
img =image.load_img(r"C:\humanfacesdata\testset\allu arjun\-1_2.jpg",grayscale=False,
                     target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
print(x.shape)
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict_classes(x)#predicting the classes
pred

(64, 64, 3)


array([0], dtype=int64)

In [20]:
index=['allu arjun','modi','obama','selena']
result=str(index[pred[0]])
result

'allu arjun'